# Modelling neutral beam injection with BBNBI

<img src='../../figures/template.png' alt='thumbnail' width='200'/>

This example shows how to generate beam ions with BBNBI5.


In [ ]:
import numpy as np
from a5py import Ascot

a5 = Ascot("ascot.h5", create=True)

print("Inputs created")